# SETTING UP ENVIRONMENT

1) <b>fork github</b> (add on github and use git pull):

*git clone git@github.com:flurinh/mt.git*

2) <b>create conda environment</b>

*conda create --name gpcr --file requirements.txt*

*conda activate gpcr*

3) <b>create (missing) folders</b>

unzip file

4) <b>check if everything works</b> (import)

In [41]:
from processing.utils import *
from processing.utils2 import *
from processing.utils3 import *
from processing.gpcrdb_soup import *
from processing.download import *
from processing.processor import *
from processing.df_to_cif import *
from analysis.analysis import *

# DATA PROCESSING

The data (and file structure) is provided in a zipfile, so you do not have to download and process everything.
This walkthrough shows how to (theoretically) build from scratch and/or update your outdated data.

## DOWNLOADING / UPDATING METADATA

With metadata we primarily refer to the [gpcrdb](https://gpcrdb.org) [GPCR structure table](https://gpcrdb.org/structure/) containing a curated list of gpcrs (their name, uniprot id etc) - new entries in this list may automatically be updated!

In our code we use a dedicated downloading class which handles both the <b>table download</b> and the <b>cif file download</b> of all missing structures. The table is saved to the subdirectory "*data/gpcrdb/*", the cif (or pdb) files are put into *data/mmcif/*.

In [14]:
D = Download(fileformat='cif')
D.download_table(reload=True)  # without reload-flag set to True (i.e. file exists) we do not re-download
# since these are unprocessed files, existing files are NOT RE-DOWNLOADED
# however we "reference" the gpcrdb gpcr structure table to see if we have missing structures, and download those
D.download_pdbs()

  0%|                                                                                                                              | 0/1 [00:00<?, ?it/s]

writing gpcrdb table to file: data/data_table.pkl
Reloading pdb files...
Missing pdbs: ['1F88']


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.44it/s]


## DOWNLOADING MORE (META)DATA

Next we initialize our processing class, creatively named ***CifProcessor***, which includes all the functions used to gather more metainformation; Such as download data from **sifts mapping** (aka the mapping of the sequence of the pdb structure against the related uniprot gene sequence), the **(generic residue) numbering** from gpcrdb, as well as does the calculation of consecutive **psi-** and **phi-angles** and **assigns the generic residue numbers** for both the gpcr or (in complexes) the gprotein. (This of course includes alignment functions and many more). Last and not least the common data format in this processor class, as we will see, are python **DataFrames**.

Let's start slow by initializing the processor:

In [2]:
p = CifProcessor()

Next we load 3 datatables (referred to as metainfo): sifts mapping ('p.mappings'), table with uniprot ids and corresponding generic residue numbers ('p.numbering') and our previously downloaded data structure table ('p.table'). 

In [3]:
p.read_pkl_metainfo()

In [19]:
p.mappings.head(3)

,identifier,name,mappings,PDB,uniprot
0,D9IEF7_BPT4,D9IEF7_BPT4,"{'entity_id': 1, 'chain_id': 'A', 'start': {'a...",6LUQ,D9IEF7
1,DRD2_HUMAN,DRD2_HUMAN,"{'entity_id': 1, 'chain_id': 'A', 'start': {'a...",6LUQ,P14416
2,DRD2_HUMAN,DRD2_HUMAN,"{'entity_id': 1, 'chain_id': 'A', 'start': {'a...",6LUQ,P14416


In [20]:
p.numbering.head(3)

,PDB,identifier,family,numbering
0,6LUQ,drd2_human,001_001_004_002,"[{'sequence_number': 1, 'amino_acid': 'M', 'pr..."
1,5L7D,smo_human,006_001_001_011,"[{'sequence_number': 1, 'amino_acid': 'M', 'pr..."
2,5TUD,5ht2b_human,001_001_001_007,"[{'sequence_number': 1, 'amino_acid': 'M', 'pr..."


In [28]:
p.table.head(3)

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function
0,STE2,STE2,D1(Ste2-likefungalpheromone),strainATCC204508/S288c,cryo-EM,7AD3,3.5,A,Active,-,GPa1family,αPa1,Agonist
1,OPRM,Opioid,A(Rhodopsin),Mouse,X-ray,4DKL,2.8,A,Inactive,7,-,-,Antagonist
2,OXYR,Vasopressinandoxytocin,A(Rhodopsin),Human,X-ray,6TPK,3.2,A,Inactive,24,-,-,Antagonist


This next step takes a very long time (**~30 minutes**), depending on if we make calls to download generic residue numbers (for uniprot ids) from [gpcrdb](https://docs.gpcrdb.org/web_services.html). (Of course if you have new samples, setting *reload_numbering* to *False* will still mean you are going to update your data to include it.) If you have an existing table, **NOT RELOADING IS ADVISED**!

In [29]:
p.make_metainfo(reload_numbering=False, reload_mapping=False, overwrite=True)

557it [00:00, 5354.56it/s]


## CREATING RAW DATAFRAMES

To start processing files from scratch you may run this next line, it will **DELETE** all pickle files (*.pkl*, the format of the dataframes) in your specified folder:

*p.del_pkl(folder='data/raw/')*

So far we only have a folder with unprocessed cif files - data is in text format. Very slow to process etc, so we first create the base structure of our dataset: a list of dataframes, **one dataframe per structure**. (During the loading process we also calculate the psi, phi and omega angles for all residues in the structure.) These are then saved (without any further processing to the subdirectory *data/raw/*.

In [30]:
p.make_raws(overwrite=False)

557it [00:00, 29309.45it/s]


Setting overwrite to False will skip all processed samples! If it is set to True, you process all your samples and overwrite the existing data.

# GENERIC RESIDUE NUMBERS

As mentioned, the dataformat we use (after loading the cif files) are DataFrames (implements a variety of SQL methods and provides easy pythonic access to big datatables). Now each of our structures corresponds to a single DataFrame in the main data container in our processor class: p.dfl (dataframe list), for easier access and search it is paired with a list containing only the respective pdb name, p.dfl_list.

In [33]:
p.dfl

[]

What happened? Well currently we have no new structures (since a few days ago when I last updated the data), thus no new structures were processed and added to the data list. To load our data:

In [4]:
p.read_pkl(mode='', folder='data/raw/')

Reading files without generic numbers...


Let's have a look at the first entry:

In [38]:
p.dfl[10].head(3)

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,Cartn_x,Cartn_y,Cartn_z,phi,omega,psi,identifier,label_comp_sid
0,2J4Y,ATOM,A,A,2,1,MET,4,N,N,-32.043,16.427,-13.796,-144.390068,179.597085,-176.94101,OPSD_BOVIN,M
1,2J4Y,ATOM,A,A,2,1,MET,5,CA,C,-31.142,16.987,-12.785,-144.390068,179.597085,-176.94101,OPSD_BOVIN,M
2,2J4Y,ATOM,A,A,2,1,MET,6,C,C,-29.663,16.868,-13.200,-144.390068,179.597085,-176.94101,OPSD_BOVIN,M


In [39]:
p.dfl_list[10]

'2J4Y'

In [7]:
p.table

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function
0,STE2,STE2,D1(Ste2-likefungalpheromone),strainATCC204508/S288c,cryo-EM,7AD3,3.5,A,Active,-,GPa1family,αPa1,Agonist
1,OPRM,Opioid,A(Rhodopsin),Mouse,X-ray,4DKL,2.8,A,Inactive,7,-,-,Antagonist
2,OXYR,Vasopressinandoxytocin,A(Rhodopsin),Human,X-ray,6TPK,3.2,A,Inactive,24,-,-,Antagonist
3,CCR2,Chemokine,A(Rhodopsin),Human,X-ray,5T1A,2.8,A,Inactive,30,-,-,AllostericantagonistAntagonist
4,CNR2,Cannabinoid,A(Rhodopsin),Human,X-ray,5ZTY,2.8,A,Inactive,6,-,-,Antagonist
...,...,...,...,...,...,...,...,...,...,...,...,...,...
552,NPY2R,NeuropeptideY,A(Rhodopsin),Human,X-ray,7DDZ,2.8,A,Inactive,1,-,-,Antagonist
553,NTR1,Neurotensin,A(Rhodopsin),Rat,cryo-EM,7L0P,4.1,C,Active,100,Gi/o,αi1,Agonist
554,ADA2C,Adrenoceptors,A(Rhodopsin),Human,X-ray,6KUW,2.8,A,Inactive,1,-,-,Antagonist
555,NTR1,Neurotensin,A(Rhodopsin),Rat,cryo-EM,7L0Q,4.3,C,Active,100,Gi/o,αi1,Agonist


## FILTERING, REDUCED TABLES AND PDB LISTS

Creating filters is easy with the processor and can primarily be done in two ways. The first is by applying the class function *p.make_filter()*:

In [5]:
f_act = p.make_filter(State='Active')

The implemented filter options (such as filtering by states etc) with examples in brackets are:
- Species  (Human, Bovine)
- State  (Active, Inactive, Intermediate)
- Cl  (Rhodopsin, A, ...)
- Family  (Gi/o, Gs, ...)
- Subtype  (αi1, ...)
- Resolution  (<=3.5)
- Function  (Agonist, Antagonist)
- gprotein  (filters out all complexes where not both Family and Subtype are within a predifined set of regular gproteins)
- pdb_ids  (eg. \['7AD3', '4DKL'\] -> creates a filter only containing the listed pdbs if present in our metainfo table) 

Filters may be "applied", where we effectively remove all data except the ones present in the filter (simultaneously being removed from both *p.dfl* and *p.dfl_list*.

In [ ]:
p.apply_filter(f_act)

The second option and this is what happens behind the scene of the *p.make_filter()* function: **QUERIES** (SQL)

In [8]:
f_act = f_act[f_act['Method']=='cryo-EM']

In [10]:
f_act.head(3)

,uniprot(gene),receptor family,Cl.,Species,Method,PDB,Resolution,Preferred Chain,State,Degree active %,Family,Subtype,Function
0,STE2,STE2,D1(Ste2-likefungalpheromone),strainATCC204508/S288c,cryo-EM,7AD3,3.5,A,Active,-,GPa1family,αPa1,Agonist
6,5HT1B,5-Hydroxytryptamine,A(Rhodopsin),Human,cryo-EM,6G79,3.8,S,Active,100,Gi/o,αo,Agonist
8,PTH1R,Parathyroidhormone,B1(Secretin),Human,cryo-EM,6NBI,4.0,R,Active,100,Gs,αs,Agonist


So filters really are just filtered metainfotables and since they are easily interchangable / by extension also pdb lists. 

## GPCR GENERIC RESIDUE NUMBERS

The tricky thing about assigning generic residue numbers to structures is that many times the residue sequence of the structure is not exactly the same as that of the *correct, corresponding gene*, further a structure may include many different chains, include fused parts, be in a complex with gproteins etc

We got the **generic residue number** from [gpcrdb](https://docs.gpcrdb.org/web_services.html), now we want to allign the uniprot sequence to the structure's sequence - this is why we got the **sifts mapping** (it's a crude alignment where we have pairs of starting and end points of intervals that align of both sequences). Unfortunately the sifts is often not correct - there are often shifts or other miss-alignments withing a given sifts interval, sometimes the sifts interval do not even have equal length... In cases where this happens we do sequence alignment ourselves.

The **DATAFLOW** is as follows:
- p.table contains the "preferred chain" information that tells us which chain in the pdb is the gpcr.
- p.mapping contains the sifts mapping (we check if the interval length is correct).
- We get the uniprot sequence.
- We use SQL to quickly produce new columns in our dataframe - very much like *label_seq_id* (which corresponds to the autogenerated sequence id of each atom (of its residue), but instead gives the position (if said atom/residue is present in the gene) within the uniprot sequence...
- .. as well as the corresponding uniprot residue name.
- We check, if there are (too) many non-matching residues of the alignment between structure and uniprot sequence and..
- .. should there be too many too many non-matching residues, we, instead of using the sifts data to align structure to uniprot, do a sequence alignment.
- Lastly, given the uniprot sequence number we can look up the generic residue numbers in p.numbering

In [11]:
p.allow_exception=False  # default

If set to True, this flag makes the processor skip any structures that throw errors, may be useful if you have "dirty data", but don't want to go through the pain of sorting out anything that won't get processed.

To start from scratch let's delete the processor (and all data loaded into memory) ... 

In [12]:
del p

... and reinitialize it, quickly, in 3 lines:

In [ ]:
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='', folder='data/raw/')

Before (finally) running the generic residue assignment a quick explanation of the function inputs:
- f: filter, this is a dataframe gained from filtering the p.table
- pdb_ids: a list of pdbs to process (optional; an empty list means we use the filter)
- overwrite: if set to True, after processing all structures get saved (overwriting existing files)
- folder: location where the processed files get stored

You need to provide a filter-like input, so the processor knows which data it should assign generic residue numbers to. Imagine loading all the data from the raw folder and being interested in *Class A (Rhodopsin)* of *humans* structures only... Notably, providing a list of ***pdb_ids*** **overwrites whatever** ***filter*** you give it. If you want **NO FILTER** (i.e. process all your raw structures) just use ***f=p.table*** (not applying any filter at all).

In [43]:
p.assign_generic_numbers_r(f=p.table, pdb_ids=[], overwrite=True, folder='data/processed/')

  0%|                                                                                                                            | 0/557 [00:00<?, ?it/s]


5K2A P29274
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False

5K2A P29274
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False
Final Error Check...

  0%|▏                                                                                                                   | 1/557 [00:00<08:33,  1.08it/s]


intervals : []
Error Status: False
assigned dfl[167] generic residue numbers for the receptor...

6KUY P08913
Found error!
6KUY: Using uniprot (looking up P08913) as a reference, due to missmatch in SIFTS!
Looking up uniprot fasta sequence for ADA2A_HUMAN
start pdb: 1
start uni: 35

6KUY P08913
Found error!
6KUY: Using uniprot (looking up P08913) as a reference, due to missmatch in SIFTS!
Looking up uniprot fasta sequence for ADA2A_HUMAN
start pdb: 274
start uni: 335

6KUY P08913
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False
Final Error Check...

  0%|▍                                                                                                                   | 2/557 [00:02<10:54,  1.18s/it]

...

Final Error Check...
intervals : []
Error Status: False
assigned dfl[518] generic residue numbers for the receptor...

5IU7 P29274
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False

5IU7 P29274
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False
Final Error Check...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 556/557 [12:42<00:01,  1.27s/it]


intervals : []
Error Status: False
assigned dfl[161] generic residue numbers for the receptor...

5ZKB P08172
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False

5ZKB P08172
Trying to assign error free uniprotlabels based on SIFTS!
intervals : []
Error Status: False
Final Error Check...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 557/557 [12:43<00:00,  1.37s/it]


intervals : []
Error Status: False
assigned dfl[249] generic residue numbers for the receptor...
Writing files with generic numbers on receptors.


The processed dataframes are safed to the specified folder (*data/processed/*, and the file is named '*\<pdb id>_r.pkl*'.

## GPROTEIN GENERIC RESIDUE NUMBERS

Very similar to how assigning generic residue numbers to receptors work, however gproteins are (EVEN) more difficult to number due to a few reasons:
- First, gpcrdb does not provide a webservice to query generic position numbers for gproteins.
- Second, we do not know which, if any at all, chain of the pdb structure is a gprotein.
- Third, there're gproteins in the form of helix-5-like-peptides fused to the gpcr, chimeras, mini-Gproteins, etc
- And of course all of the problems we faced with gpcrs in terms of aligning them to the uniprot sequence. (Just worse since often there are no sifts mappings provided at all.)

How did we solve these issues?
- First issue: This could be easily solved, since the number of gproteins is far less than that of receptors, and gpcrdb does provide an old excel sheet with generic residue numbers for all of them (the file was downloaded and manually updated from .xls to .xlsx). The file is stored in "*data/alignments/residue_table.xlsx*". 
- Second issue: since there is a limited number of gproteins we do sequence alignments of ALL chains to ALL gprotein (uniprot) sequences and, using a scoring function as well as a threshold, we detect the BEST gprotein fit if it is present.
- Third issue: This is not solved per-se, but generally two facts somewhat make our solution work: using alignments to find gproteins does make the type of gprotein we're looking at irrelevant, since IF we manage to find a good enough alignment we simply use that; Also many gproteins are similar, so even if not the correct gprotein is picked from the alignment (often happens in chimeras) due to their overall similarity the results seem consistently fine.
- And... Pain.

Quick note: The function expects as an input files that have already been processed by assigning generic residue numbers to the receptor (this is useful to check if what we're trying to label is the gpcr chain). Thus you should always either load data from the processed folder with the flag *mode='r'* or if you start by processing raws first run "*assign_generic_numbers_r*".

To make sure everything you want to assign gprotein-generic-residue-numbers to, let's reinitialize/load our data from the correct folder. Note that the flag 'mode' in the *p.read_pkl()* function can be:
- '' / an empty string: This loads the raw dataframes from the raw folder without gen. position numbers (requires *folder='data/raw/'*)!
- 'r': This loads the dataframes where generic position numbers have been assigned for the receptors (but not the gproteins)!
- 'rg': This loads the dataframes where generic position numbers have been assigned for both gpcr and gprotein!

In [18]:
del p
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='r', folder='data/processed/')
f_act = p.make_filter(State='Active', Cl='Rhodopsin', gprotein=True)
f_act = f_act[f_act['Method']=='cryo-EM']

Reading files with generic numbers on receptors.


In [19]:
p.assign_generic_numbers_g(f=f_act, pdb_ids=[], overwrite=False, folder='data/processed/', fill_H5=True)

  0%|                                                                                                                             | 0/67 [00:00<?, ?it/s]

assinging res_nums to gprot of 6WHA
No mapping (no uniprot-seq) information found! ====> Assigning labels ourself!
_assign_res_nums_mini_g
start_uni 12 > end_uni 58
start_pdb 5 > end_pdb 51
mini g setting index += 1
Found corr. gen num! idx_uni: 11 <> idx_seq: 5 <> gen pos G.HN.29
Found corr. gen num! idx_uni: 12 <> idx_seq: 6 <> gen pos G.HN.30
Found corr. gen num! idx_uni: 13 <> idx_seq: 7 <> gen pos G.HN.31
Found corr. gen num! idx_uni: 14 <> idx_seq: 8 <> gen pos G.HN.32
Found corr. gen num! idx_uni: 15 <> idx_seq: 9 <> gen pos G.HN.33
Found corr. gen num! idx_uni: 16 <> idx_seq: 10 <> gen pos G.HN.34
Found corr. gen num! idx_uni: 17 <> idx_seq: 11 <> gen pos G.HN.35
Found corr. gen num! idx_uni: 18 <> idx_seq: 12 <> gen pos G.HN.36
Found corr. gen num! idx_uni: 19 <> idx_seq: 13 <> gen pos G.HN.37
Found corr. gen num! idx_uni: 20 <> idx_seq: 14 <> gen pos G.HN.38
Found corr. gen num! idx_uni: 21 <> idx_seq: 15 <> gen pos G.HN.39
Found corr. gen num! idx_uni: 22 <> idx_seq: 16 <> g

Found corr. gen num! idx_uni: 330 <> idx_seq: 183 <> gen pos G.hgh4.21
Found corr. gen num! idx_uni: 331 <> idx_seq: 184 <> gen pos G.H4.01
Found corr. gen num! idx_uni: 332 <> idx_seq: 185 <> gen pos G.H4.02
Found corr. gen num! idx_uni: 333 <> idx_seq: 186 <> gen pos G.H4.03
Found corr. gen num! idx_uni: 334 <> idx_seq: 187 <> gen pos G.H4.04
Found corr. gen num! idx_uni: 335 <> idx_seq: 188 <> gen pos G.H4.05
Found corr. gen num! idx_uni: 336 <> idx_seq: 189 <> gen pos G.H4.06
Found corr. gen num! idx_uni: 337 <> idx_seq: 190 <> gen pos G.H4.07
Found corr. gen num! idx_uni: 338 <> idx_seq: 191 <> gen pos G.H4.08
Found corr. gen num! idx_uni: 339 <> idx_seq: 192 <> gen pos G.H4.10
Found corr. gen num! idx_uni: 340 <> idx_seq: 193 <> gen pos G.H4.11
Found corr. gen num! idx_uni: 341 <> idx_seq: 194 <> gen pos G.H4.12
Found corr. gen num! idx_uni: 342 <> idx_seq: 195 <> gen pos G.H4.13
Found corr. gen num! idx_uni: 343 <> idx_seq: 196 <> gen pos G.H4.14
Found corr. gen num! idx_uni: 34

  1%|█▋                                                                                                                   | 1/67 [00:01<02:04,  1.89s/it]


assinging res_nums to gprot of 7L1U
No mapping (no uniprot-seq) information found! ====> Assigning labels ourself!
_assign_res_nums_mini_g
start_uni 11 > end_uni 60
start_pdb 2 > end_pdb 51
mini g setting index += 1
Found corr. gen num! idx_uni: 10 <> idx_seq: 2 <> gen pos G.HN.28
Found corr. gen num! idx_uni: 11 <> idx_seq: 3 <> gen pos G.HN.29
Found corr. gen num! idx_uni: 12 <> idx_seq: 4 <> gen pos G.HN.30
Found corr. gen num! idx_uni: 13 <> idx_seq: 5 <> gen pos G.HN.31
Found corr. gen num! idx_uni: 14 <> idx_seq: 6 <> gen pos G.HN.32
Found corr. gen num! idx_uni: 15 <> idx_seq: 7 <> gen pos G.HN.33
Found corr. gen num! idx_uni: 16 <> idx_seq: 8 <> gen pos G.HN.34
Found corr. gen num! idx_uni: 17 <> idx_seq: 9 <> gen pos G.HN.35
Found corr. gen num! idx_uni: 18 <> idx_seq: 10 <> gen pos G.HN.36
Found corr. gen num! idx_uni: 19 <> idx_seq: 11 <> gen pos G.HN.37
Found corr. gen num! idx_uni: 20 <> idx_seq: 12 <> gen pos G.HN.38
Found corr. gen num! idx_uni: 21 <> idx_seq: 13 <> gen

...

 99%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 66/67 [00:50<00:00,  1.49it/s]


Found corr. gen num! idx_uni: 282 <> idx_seq: 283 <> gen pos G.HG.15
Found corr. gen num! idx_uni: 283 <> idx_seq: 284 <> gen pos G.HG.16
Found corr. gen num! idx_uni: 284 <> idx_seq: 285 <> gen pos G.HG.17
Found corr. gen num! idx_uni: 285 <> idx_seq: 286 <> gen pos G.hgh4.01
Found corr. gen num! idx_uni: 286 <> idx_seq: 287 <> gen pos G.hgh4.02
Found corr. gen num! idx_uni: 287 <> idx_seq: 288 <> gen pos G.hgh4.03
Found corr. gen num! idx_uni: 288 <> idx_seq: 289 <> gen pos G.hgh4.04
Found corr. gen num! idx_uni: 289 <> idx_seq: 290 <> gen pos G.hgh4.05
Found corr. gen num! idx_uni: 290 <> idx_seq: 291 <> gen pos G.hgh4.06
Found corr. gen num! idx_uni: 291 <> idx_seq: 292 <> gen pos G.hgh4.07
Found corr. gen num! idx_uni: 292 <> idx_seq: 293 <> gen pos G.hgh4.08
Found corr. gen num! idx_uni: 293 <> idx_seq: 294 <> gen pos G.H4.01
Found corr. gen num! idx_uni: 294 <> idx_seq: 295 <> gen pos G.H4.02
Found corr. gen num! idx_uni: 295 <> idx_seq: 296 <> gen pos G.H4.03
Found corr. gen n

Found corr. gen num! idx_uni: 203 <> idx_seq: 203 <> gen pos G.hfs2.05
Found corr. gen num! idx_uni: 204 <> idx_seq: 204 <> gen pos G.hfs2.06
Found corr. gen num! idx_uni: 205 <> idx_seq: 205 <> gen pos G.hfs2.07
Found corr. gen num! idx_uni: 206 <> idx_seq: 206 <> gen pos G.S2.01
Found corr. gen num! idx_uni: 207 <> idx_seq: 207 <> gen pos G.S2.02
Found corr. gen num! idx_uni: 208 <> idx_seq: 208 <> gen pos G.S2.03
Found corr. gen num! idx_uni: 209 <> idx_seq: 209 <> gen pos G.S2.04
Found corr. gen num! idx_uni: 210 <> idx_seq: 210 <> gen pos G.S2.05
Found corr. gen num! idx_uni: 211 <> idx_seq: 211 <> gen pos G.S2.06
Found corr. gen num! idx_uni: 212 <> idx_seq: 212 <> gen pos G.S2.07
Found corr. gen num! idx_uni: 213 <> idx_seq: 213 <> gen pos G.S2.08
Found corr. gen num! idx_uni: 214 <> idx_seq: 214 <> gen pos G.s2s3.01
Found corr. gen num! idx_uni: 215 <> idx_seq: 215 <> gen pos G.s2s3.02
Found corr. gen num! idx_uni: 216 <> idx_seq: 216 <> gen pos G.S3.01
Found corr. gen num! idx

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 67/67 [00:50<00:00,  1.32it/s]


Found corr. gen num! idx_uni: 300 <> idx_seq: 300 <> gen pos G.HG.08
Found corr. gen num! idx_uni: 301 <> idx_seq: 301 <> gen pos G.HG.09
Found corr. gen num! idx_uni: 302 <> idx_seq: 302 <> gen pos G.HG.10
Found corr. gen num! idx_uni: 303 <> idx_seq: 303 <> gen pos G.HG.11
Found corr. gen num! idx_uni: 304 <> idx_seq: 304 <> gen pos G.HG.12
Found corr. gen num! idx_uni: 305 <> idx_seq: 305 <> gen pos G.HG.13
Found corr. gen num! idx_uni: 306 <> idx_seq: 306 <> gen pos G.HG.14
Found corr. gen num! idx_uni: 307 <> idx_seq: 307 <> gen pos G.HG.15
Found corr. gen num! idx_uni: 308 <> idx_seq: 308 <> gen pos G.HG.16
Found corr. gen num! idx_uni: 309 <> idx_seq: 309 <> gen pos G.HG.17
Found corr. gen num! idx_uni: 310 <> idx_seq: 310 <> gen pos G.hgh4.01
Found corr. gen num! idx_uni: 311 <> idx_seq: 311 <> gen pos G.hgh4.02
Found corr. gen num! idx_uni: 312 <> idx_seq: 312 <> gen pos G.hgh4.03
Found corr. gen num! idx_uni: 313 <> idx_seq: 313 <> gen pos G.hgh4.04
Found corr. gen num! idx_

In [20]:
len(p.dfl)

557

In [21]:
p.apply_filter(f_act)

In [22]:
len(p.dfl)

67

In [27]:
df = p.dfl[10]

In [28]:
df[df['gprot_pos']!=''].head(10)

,PDB,group_PDB,auth_asym_id,label_asym_id,label_seq_id,auth_seq_id,label_comp_id,id,label_atom_id,type_symbol,...,identifier,label_comp_sid,label_2_uni,gen_pos,gen_pos1,gen_pos2,uniprot_comp_sid,gprot_pos,uniprot_comp_id,fam_comp_id
1,6LFM,ATOM,A,A,2,3,CYS,2,CA,C,...,CXCR2_HUMAN,C,2,,0.0,0.0,,G.HN.09,C,C
7,6LFM,ATOM,A,A,3,4,THR,8,CA,C,...,CXCR2_HUMAN,T,3,,0.0,0.0,,G.HN.10,T,T
14,6LFM,ATOM,A,A,4,5,LEU,15,CA,C,...,CXCR2_HUMAN,L,4,,0.0,0.0,,G.HN.11,L,L
22,6LFM,ATOM,A,A,5,6,SER,23,CA,C,...,CXCR2_HUMAN,S,5,,0.0,0.0,,G.HN.30,S,S
28,6LFM,ATOM,A,A,6,7,ALA,29,CA,C,...,CXCR2_HUMAN,A,6,,0.0,0.0,,G.HN.31,A,A
33,6LFM,ATOM,A,A,7,8,GLU,34,CA,C,...,CXCR2_HUMAN,E,7,,0.0,0.0,,G.HN.32,E,E
42,6LFM,ATOM,A,A,8,9,ASP,43,CA,C,...,CXCR2_HUMAN,D,8,,0.0,0.0,,G.HN.33,D,D
50,6LFM,ATOM,A,A,9,10,LYS,51,CA,C,...,CXCR2_HUMAN,K,9,,0.0,0.0,,G.HN.34,K,K
59,6LFM,ATOM,A,A,10,11,ALA,60,CA,C,...,CXCR2_HUMAN,A,10,,0.0,0.0,,G.HN.35,A,A
64,6LFM,ATOM,A,A,11,12,ALA,65,CA,C,...,CXCR2_HUMAN,A,11,,0.0,0.0,,G.HN.36,A,A


Assigning generic residue numbers to anything but **active complexes** is a waste of computational time and potentially errenous, since looking for the *BEST* alignment of any gprotein to *anything* in your structure **may result in things getting labelled that are NOT the gprotein** (there is a threshold on the alignment score, but I've not checked what happens... so I take no responsibility ^^). 

The fill_H5 flag was used since many of the H5-helices did not get labelled - especially in weird active complexes where not the full H5 is present.

## Modifying *.cif* files

In [32]:
structure = p.dfl[10]
df = spread_gen_pos_to_residue_atoms(structure)
doc = write_cif_with_gen_seq_numbers(df, replace_idx = -7)
doc.write_file('data/modified/'+df['PDB'].iloc[0]+'_modified.cif')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:02<00:00,  2.86it/s]


The only thing to mention here is that cif (or pdb files in general) and their "visualization" in pymol are fixed, and we merely choose a column to overwrite (e.g. 'b-factors') with the generic position numbers which can be labelled in pymol.
This works reasonably well for gpcr generic position numbers (since they can be represented as a float, however for gproteins the type of *string* makes this crude method fail (assigns a 0.0 instead of 'G.H5.26'), another column could be the 'auth_seq_id'...

# ANALSYSIS

## DISTANCE ANALYSIS

## HELICES - ANGLE ANALYSIS

To do helices analysis (both helices in the gprotein and gpcrs) we need to initialize a processor in 'rg' mode:

In [36]:
del p

In [37]:
p = CifProcessor()
p.read_pkl_metainfo()
p.read_pkl(mode='rg', folder='data/processed/')

Reading files with generic numbers on receptors and gproteins.


Filter the data...

In [39]:
f_act = p.make_filter(State='Active', Cl='Rhodopsin', gprotein=True)
f_act = f_act[f_act['Method']=='cryo-EM']
p.apply_filter(f_act)

In [40]:
len(p.dfl)

67

# AFFINITY/SELECTIVITY PREDCITION

## AFFINITIES

## DATALOADER / DATASET

## NEURAL NETWORK

## REVERSE NN ANALYSIS